In [1]:
# import distutils
import numpy as np
from pickle import load 
from pickle import dump
from PIL import Image 
import os
import string
import tensorflow as tf 
import pandas as pd

In [10]:
def load_file_path(filename):
    with open(filename, 'r') as file:
        text = file.read()
    return text

def map_image_to_captions(filename):
    file  = load_file_path(filename)
    lines  = file.strip().splitlines()
    image_to_caption = {}
    for line in lines:
        img, caption = line.split(',', 1)
        img = img.strip()
        caption = caption.strip()

        if img not in image_to_caption:
            image_to_caption[img] = [caption]
        else:
            image_to_caption[img].append(caption)
    return image_to_caption

def clean_captions(image_to_caption):
    translation_table = str.maketrans('', '', string.punctuation)

    for img, caps in image_to_caption.items():
        for i in range(len(caps)):
            img_caption = caps[i].replace("-", " ")
            tokens = img_caption.split()
            cleaned_words = [
                word.lower().translate(translation_table) for word in tokens
                if len(word) > 1 and word.isalpha()
            ]
            caps[i] = ' '.join(cleaned_words)
    return image_to_caption


def create_vocab(image_to_caption):
    vocab = set()
    for img, caps in image_to_caption.items():
        for caption in caps:
            vocab.update(caption.split()) 

    return vocab


def save_img_to_captions(image_to_caption, filename_to_save):
    lines = []
    for img, caps in image_to_caption.items():
        for caption in caps:
            lines.append(img + '\t' + caption)

    data = "\n".join(lines) 
    with open(filename_to_save, "w") as file:
        file.write(data)


In [ ]:
dataset_text = r'C:\Users\user\Desktop\ML\Image_Caption_Generator\flickr8k\captions.txt'

image_to_caption = map_image_to_captions(dataset_text)
print("length of dictionary = " ,len(image_to_caption))

clean_texts = clean_captions(image_to_caption)

vocabulary = create_vocab(clean_texts)
print('length of vocabulary =', len(vocabulary))

save_img_to_captions(clean_texts, 'img_caption.txt')

length of dictionary =  8092
length of vocabulary = 8405


## Feature Extraction

In [2]:
from tensorflow.keras.applications.xception import Xception #to get pre-trained model Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
# from keras.preprocessing.text import Tokenizer #for text tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense#Keras to build our CNN and LSTM
from tensorflow.keras.layers import LSTM, Embedding, Dropout
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm #to check loop progress
# tqdm().pandas()

In [5]:
model = Xception(include_top=False, pooling = 'avg')


In [6]:
def extract_features(directory):
    features = {}
    for pic in tqdm(os.listdir(directory)):
        file_path = os.path.join(directory,pic)

        img = Image.open(file_path)
        img = img.resize((299,299))
        img_array = np.expand_dims(np.array(img) / 127.5 - 1.0, axis=0)
        
        features_vector = model.predict(img_array)
        features[pic] = features_vector
    return features

In [7]:
dataset_images = r'C:\Users\user\Desktop\ML\Image_Caption_Generator\flickr8k\Images'
features = extract_features(dataset_images)

with open("features.p", "wb") as file:
    dump(features,file)

  0%|          | 0/8091 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


  0%|          | 1/8091 [00:01<4:27:44,  1.99s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step


  0%|          | 2/8091 [00:02<2:27:19,  1.09s/it]

...

100%|█████████▉| 8089/8091 [1:22:14<00:01,  1.58it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step


100%|█████████▉| 8090/8091 [1:22:14<00:00,  1.68it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step


100%|██████████| 8091/8091 [1:22:15<00:00,  1.64it/s]


In [ ]:
with open("features.p", "rb") as file:
    img_features = load(file)


## Tokenizing the Captions

In [12]:
df_tok = pd.read_csv(r'C:\Users\user\Desktop\ML\Image_Caption_Generator\img_caption.txt', delimiter='\t')
print(df_tok.head(6))

                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   
5  1001773457_577c3a7d70.jpg   

                                             caption  
0  child in pink dress is climbing up set of stai...  
1                    girl going into wooden building  
2         little girl climbing into wooden playhouse  
3   little girl climbing the stairs to her playhouse  
4  little girl in pink dress going into wooden cabin  
5             black dog and spotted dog are fighting  


In [13]:
captions_on_tok =df_tok['caption'].tolist() 

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_on_tok)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in captions_on_tok)
tokenizer.texts_to_sequences([captions_on_tok[1]])[0]

## Train-test split

images = df_tok['image'].unique().tolist()
nimages = len(images)

split_index = round(0.85*nimages)
train_images = images[:split_index]
val_images = images[split_index:]

train = df_tok[df_tok['image'].isin(train_images)]
test = df_tok[df_tok['image'].isin(val_images)]

train.reset_index(inplace=True,drop=True)
test.reset_index(inplace=True,drop=True)